# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [2]:
from google.colab import files
uploaded = files.upload()

Saving lenta_reduced.txt to lenta_reduced.txt


In [1]:
!pip install razdel
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML
from nltk.tokenize import sent_tokenize
from collections import Counter
import nltk
nltk.download('punkt')

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

news = open('lenta_reduced.txt').read()

print("Length of corpus -", len(news))

sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news[10:]:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence, 2))
    trigrams_news.update(ngrammer(sentence, 3))

print('Most common: ', trigrams_news.most_common(10))


print("Length of unigrams_news: {}".format(len(unigrams_news)))
print("Length of biigrams_news: {}".format(len(bigrams_news)))
print("Length of trigrams_news: {}".format(len(trigrams_news)))
# создадим матрицу вероятностей перейти из 1 слова в два других
matrix_news = np.zeros((len(bigrams_news), len(unigrams_news)))
id2word_news = list(unigrams_news)
id2bigram_news = list(bigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}
bigram2id_news = {word:i for i, word in enumerate(id2bigram_news)}

# вероятность перейти из слова к триграмме расчитываем точно также
for ngram in trigrams_news:
    #print(ngram, trigrams_news[ngram], unigrams_news[word1])
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2
    matrix_news[bigram2id_news[bigram]][word2id_news[word3]] = trigrams_news[ngram]/bigrams_news[bigram]





[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Length of corpus - 574769
Most common:  [('<start> <start> в', 328), ('<start> <start> по', 292), ('<start> <start> как', 203), ('<start> <start> на', 98), ('<start> <start> об', 75), ('<start> об этом', 71), ('<start> <start> однако', 70), ('<start> по словам', 68), ('<start> как сообщает', 63), ('в связи с', 56)]
Length of unigrams_news: 19510
Length of biigrams_news: 57474
Length of trigrams_news: 69546


In [72]:
def generate(matrix, id2word, id2bigram, word2id, bigram2id, n=50, start='<start> <start>'):
    text = []
    end_added = False
    current_idx = bigram2id[start]
    
    for i in range(n):
        p = matrix[current_idx]
        chosen = np.random.choice(matrix.shape[1], p=p)
        
        if id2word[chosen] == '<end>':
            if not end_added:
                text.append(id2word[chosen])
                end_added = True
            chosen = word2id['<start>']
        else:
            text.append(id2word[chosen])
            bigram_next = id2bigram[current_idx].split()[1:][0] + ' ' + id2word[chosen]
            #print(bigram_next)
            current_idx = bigram2id[bigram_next]
    
    return ' '.join(text)

for i in range(5):
    a = generate(matrix_news, id2word_news, id2bigram_news, word2id_news, bigram2id_news).replace('<end>', '\n')
    print(a)


как сообщает интерфакс сегодня утром около трехсот сотрудников омон вооруженных автоматами заняли центральный офис компании транснефть в москве в этом конфликте 

в настоящее время составляет около 1,1 миллиардов 

по его сведениям за последние сто лет 

на тайвань уже прибыли бригады спасателей из таиланда и сингапура 

по сообщению агентства assotiated press 
 в западной турции где 17 августа произошло приведшее к гибели более 14 тысяч человек сегодняв 11.11 местного времени 19 часов по москве и выражает соболезнование семьям погибших и раненых из глубокой грязи


In [ ]:
# корпус пришлсь сильно сократить из-за недостатка памяти, но даже с маленьким корпусом при использоваии триграмм повышается качество генерируемого текста
# для более качественной генерации нужен больший корпус

In [75]:
# перплексия
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N)

for sent in sentences_news[:10]:
    prob = []
    for ngram in ngrammer(sent, 3):
        bigram = " ".join(ngram.split()[:2])
        #print('ngram: ', ngram)
        #print('bigram: ', bigram)
        if ngram in trigrams_news and bigram in bigrams_news:
            prob.append(np.log(trigrams_news[ngram] / bigrams_news[bigram]))
        else:
            prob.append(np.log(0.00001))
    print(" ".join(sent), perplexity(prob), '\n')


<start> <start> бои у сопоцкина и друскеник закончились отступлением германцев <end> 69429.83929883369 

<start> <start> неприятель приблизившись с севера к осовцу начал артиллерийскую борьбу с крепостью <end> 99999.9999999999 

<start> <start> в артиллерийском бою принимают участие тяжелые калибры <end> 32155.49500829696 

<start> <start> с раннего утра 14 сентября огонь достиг значительного напряжения <end> 51603.09390798733 

<start> <start> попытка германской пехоты пробиться ближе к крепости отражена <end> 99999.9999999999 

<start> <start> в галиции мы заняли дембицу <end> 22029.616293703886 

<start> <start> большая колонна отступавшая по шоссе от перемышля к саноку обстреливалась с высот нашей батареей и бежала бросив парки обоз и автомобили <end> 100000.00000000029 

<start> <start> вылазки гарнизона перемышля остаются безуспешными <end> 99999.9999999999 

<start> <start> при продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей захватываются новы

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

Чтобы преодолеть проблему OOV-слов (т. е. слов, которые не встречаются в корпусе), можно создать систему с открытым словарем, выбрать фиксированный словарь (список слов), на этапе нормализации заменить в корпусе все слова не из списка на токен `<UNK>` и обучать модель так, будто `<UNK>` — обычное слово.
В случае отсутствия предварительного словаря можно создать такой словарь неявно, заменив слова в обучающих данных на `<UNK>` в зависимости от их частотности (заменяются наименее частотные).

2. Что такое сглаживание (smoothing)?

Проблема недостаточных данных (sparse data) заключается в том, что в результате обучения на конечном корпусе многие n-граммы не будут в нем встречаться и получат нулевую вероятность, хотя в действительности встречаются и должны иметь ненулевую.
Чтобы такого не происходило, придется срезать некоторую массу вероятности с некоторых более частотных n-грамм и передать n-граммам, которые не встретились в контексте. Эта модификация называется сглаживанием. Существует несколько способов сглаживания:
* сглаживание Лапласа, по которому происходит добавление 1 ко всем частотам в корпусе (метод провоцирует сильную погрешность в вычислениях)
* аддитивное сглаживание (add-k smoothing)
* откат (backoff): можно оценить вероятности N-грамм с помощью вероятностей (N-k)-грамм (0<k<N)
* сглаживание Кнесера-Нея